## 把 sft_training_data_final.jsonl 拆成測試跟驗證集的地方

In [ ]:
import json
import os
import re
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter

In [ ]:
# manually upload 'sft_training_data_final.jsonl'

In [ ]:
# ==========================================
# 設定區
# ==========================================
INPUT_FILE = 'sft_training_data_final.jsonl'
OUTPUT_DIR = 'split_dataset_smart'
RANDOM_SEED = 42

# 目標切分數量
TRAIN_SIZE = 300
VAL_SIZE = 29
TEST_SIZE = 40

# 您的 OpenAI API Key
# 策略：優先讀取環境變數，若無則嘗試讀取 Google Colab Secrets
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    try:
        from google.colab import userdata
        OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    except ImportError:
        # 非 Colab 環境，忽略
        pass
    except Exception as e:
        # 雖然是 Colab 但可能沒設 Secret，或者 Key 名稱不對
        print(f"提示: 未從 Colab Secret 讀取到 'OPENAI_API_KEY'。若有需要請在側欄 '🔑' 設定。")

# 定義職災類別標籤 (讓 LLM 從中選擇)
INCIDENT_CATEGORIES = [
    "墜落、滾落",       #
    "跌倒",             #
    "衝撞",             # (指人體以自身動力碰撞物體)
    "物體飛落",         #
    "物體倒塌、崩塌",   #
    "被撞",             # (指人體被飛來、落下、滑動之物體擊中)
    "被夾、被捲",       #
    "被切、割、擦傷",   #
    "踩踏",             #
    "溺斃",             #
    "與高溫、低溫接觸", #
    "與有害物等之接觸", # (含吸入中毒)
    "感電",             #
    "爆炸",             #
    "物體破裂",         #
    "火災",             #
    "不當動作",         #
    "交通事故",         # 合併 (公路、鐵路、船舶等)
    "其他"              # (無法歸類)
]

In [ ]:
def clean_text_for_preview(text):
    """
    使用 Regex 強力清洗：
    1. 去除所有非中文字符 (去除標點、數字、英文)
    2. 去除 '○' (去識別化符號)
    3. 去除常見無意義詞
    """
    # 僅保留中文字 (\u4e00-\u9fa5)
    text = re.sub(r'[^\u4e00-\u9fa5]', '', text)
    # 去除特定雜訊字
    text = re.sub(r'[○oO]', '', text)
    return text

In [ ]:
def get_category_from_openai(text, client):
    """
    呼叫 OpenAI API，依據官方「災害類型」代號表進行精準分類。
    """
    snippet = text[:600] # 稍微增加長度以包含更多情境

    # System Prompt: 賦予專家角色，並給予嚴格的分類指導
    # 這裡的 Prompt 設計是為了防止 LLM 混淆「媒介物」與「災害類型」
    system_prompt = """
    你是一位精通台灣職業安全法規的專家。請依據「職業災害類型分類表」對事故進行分類。

    【重要原則】
    1. 請區分「媒介物」與「災害類型」。例如：被「堆高機」撞到，災害類型應選「被撞」，而非其他。
    2. 「衝撞」指人去撞物體；「被撞」指物體來撞人。
    3. 若涉及墜落，優先選擇「墜落、滾落」。
    4. 若涉及觸電，選擇「感電」。
    """

    user_prompt = f"""
    請閱讀以下職災事故描述，並從下列標準清單中，選出最主要的一個「災害類型」：

    {json.dumps(INCIDENT_CATEGORIES, ensure_ascii=False)}

    事故描述：
    {snippet}

    請直接輸出類別名稱（例如：被夾、被捲），不要輸出任何解釋或其他文字。
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0 # 設為 0 確保分類結果穩定
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"API Error: {e}")
        return "其他"

In [ ]:
def main():
    # 檢查 API Key
    # 優先使用全域變數 (來自 os.getenv)，若無則再次嘗試讀取 env (以防設定順序問題)
    api_key = OPENAI_API_KEY or os.getenv("OPENAI_API_KEY")

    if not api_key or "sk-" not in api_key:
        print("錯誤：請設定正確的 OPENAI_API_KEY 環境變數，否則無法進行智慧分類。")
        print("提示：您可以使用 export OPENAI_API_KEY='sk-...' 來設定")
        return

    from openai import OpenAI
    client = OpenAI(api_key=api_key)

    # 1. 讀取資料
    print(f"正在讀取 {INPUT_FILE}...")
    data = []
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))

    df = pd.DataFrame(data)

    # 2. AI 智慧分類 (如果已經跑過，建議把結果存成暫存檔，省錢)
    # 這裡我們建立一個 cache 機制
    cache_file = "incident_categories_cache.json"

    if os.path.exists(cache_file):
        print("偵測到分類快取檔，正在載入...")
        with open(cache_file, 'r', encoding='utf-8') as f:
            categories = json.load(f)
    else:
        print(f"開始呼叫 OpenAI API 對 {len(df)} 筆資料進行分類 (預計花費 < $0.1 USD)...")
        categories = []
        for i, row in df.iterrows():
            if i % 10 == 0: print(f"進度: {i}/{len(df)}")
            cat = get_category_from_openai(row['input'], client)
            # 簡單清洗回傳結果 (避免 LLM 多話)
            found_cat = "其他"
            for valid_cat in INCIDENT_CATEGORIES:
                if valid_cat in cat:
                    found_cat = valid_cat
                    break
            categories.append(found_cat)

        # 存檔
        with open(cache_file, 'w', encoding='utf-8') as f:
            json.dump(categories, f, ensure_ascii=False)

    df['category'] = categories

    # 3. 顯示分類結果統計
    print("\n[AI 判斷] 職災類型分佈：")
    print(df['category'].value_counts())

    # 4. 分層切分 (Stratified Split)
    # 這是最關鍵的一步：保證 Train/Val/Test 都有各種災害

    print("\n正在進行分層切分...")

    # 第一刀：切出 Train (300)
    # 注意：如果某個類別只有 1 筆，stratify 會報錯，所以需要過濾或退化處理
    try:
        df_train, df_temp = train_test_split(
            df,
            train_size=TRAIN_SIZE,
            stratify=df['category'],
            random_state=RANDOM_SEED
        )
    except ValueError as e:
        print(f"警告：某些類別樣本太少，無法完美分層。轉為隨機切分。(錯誤: {e})")
        df_train, df_temp = train_test_split(df, train_size=TRAIN_SIZE, random_state=RANDOM_SEED)

    # 第二刀：切出 Test (40) 和 Val (29)
    try:
        df_test, df_val = train_test_split(
            df_temp,
            train_size=TEST_SIZE, # 絕對數字 40
            stratify=df_temp['category'],
            random_state=RANDOM_SEED
        )
    except ValueError:
        print("警告：剩餘資料類別分佈稀疏，第二階段轉為隨機切分。")
        df_test, df_val = train_test_split(
            df_temp,
            train_size=TEST_SIZE,
            random_state=RANDOM_SEED
        )

    # 5. 輸出結果與儲存
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    def save_split(df_split, name):
        path = os.path.join(OUTPUT_DIR, name)
        # 移除暫存的 category 欄位，保持原始格式
        records = df_split.drop(columns=['category']).to_dict('records')
        with open(path, 'w', encoding='utf-8') as f:
            for r in records:
                f.write(json.dumps(r, ensure_ascii=False) + '\n')

        # 統計該 Split 的分佈給使用者看
        print(f"\n檔案: {name} ({len(records)} 筆)")
        print("分佈概況:", df_split['category'].value_counts().to_dict())

    save_split(df_train, 'train.jsonl')
    save_split(df_val, 'val.jsonl')
    save_split(df_test, 'test.jsonl')

    print(f"\n全部完成！檔案已儲存於 {OUTPUT_DIR}/")

In [ ]:
if __name__ == "__main__":
    main()

正在讀取 sft_training_data_final.jsonl...
偵測到分類快取檔，正在載入...

[AI 判斷] 職災類型分佈：
category
墜落、滾落       199
物體倒塌、崩塌      36
被夾、被捲        27
物體飛落         24
跌倒           20
被撞           13
其他           10
火災            8
感電            8
交通事故          5
衝撞            5
爆炸            4
溺斃            4
與有害物等之接觸      3
與高溫、低溫接觸      2
被切、割、擦傷       1
Name: count, dtype: int64

正在進行分層切分...
警告：某些類別樣本太少，無法完美分層。轉為隨機切分。(錯誤: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.)
警告：剩餘資料類別分佈稀疏，第二階段轉為隨機切分。

檔案: train.jsonl (300 筆)
分佈概況: {'墜落、滾落': 162, '物體倒塌、崩塌': 27, '被夾、被捲': 26, '物體飛落': 21, '跌倒': 14, '被撞': 10, '其他': 8, '火災': 7, '感電': 6, '衝撞': 5, '溺斃': 4, '交通事故': 4, '與有害物等之接觸': 3, '爆炸': 2, '與高溫、低溫接觸': 1}

檔案: val.jsonl (29 筆)
分佈概況: {'墜落、滾落': 16, '跌倒': 4, '物體倒塌、崩塌': 4, '物體飛落': 2, '爆炸': 1, '感電': 1, '被撞': 1}

檔案: test.jsonl (40 筆)
分佈概況: {'墜落、滾落': 21, '物體倒塌、崩塌': 5, '跌倒': 2, '被撞': 2, '其他': 2, '被夾、被捲': 1, '交通事故': 1, '物體飛落': 1, '與高溫、低溫接觸'